## RSA 

#### Introduction
RSA is an algorithm used to encrypt and decrypt messages.  The algorithm uses public and private keys along with modular arithmetic and prime factorization to make this type of encryption impossible to decode with large numbers.  

To begin, you'll need to create a public key, which will give you two integers; e and n.  Choose two different prime numbers; p and q.  To get n multiple p and q. (n = p*q).  e is a number that is relatively prime to (p-1)*(q-1).  Once you've caluclated e and n, you have your public key, which you can send to anyone!  It's important to note here that it's very difficult, for large numbers for someone to determine the values of p and q.  

To encode a message, convert each character in your message to some numerical value.  For this project we've chosen ASCII.  For each numberic value from your message (m), m^e mod n, where e and n are the values from your public key.  Now you will have a list of each of your encoded values you calcluated using your public key.  Now you can send the encrypted message back to whomever gave you the public key and they will be able to decode your message. 

To decode the message you will need to create a private key, which will give you two integers; d and n.  d is the inverse of e mod (p-1)(q-1).  For each numberic value from your encoded message (c), c^d mod n, where d and n are the values from your private key.  This will give you a list of numbers that coorespond to their orignal (for this project) ASCII characters.

#### Basic tool set

These are functions that you'll need to pre-process the messages before the messages are encoded and decoded by the RSA algorithm. Thats the reason we will be defining them first.



In [3]:
#convert a string into a list of ascii characters
def Convert_Text_ascii(_string):
    integer_list = [] #creates an empty list
    
    for i in _string: #loops through each character in the string
        ascii_ord = ord(i) #converts each character in the string to the respective ascii number
        integer_list.append(ascii_ord) #adds the assoicated ascii number to a list
    
    return integer_list #returns the list of ascii numbers as a list

Convert_Text_ascii("Hello World!")

[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 33]

In [4]:
#converts a list of ascii numbers into a string of their respective characters
def Convert_Num_ascii(_list):
    _string = '' #creates an empty string
    
    for i in _list: #loops through each number in the list
        ascii_chr = chr(i) #assigns each number from the list to its respective character
        _string += ascii_chr #adds the converted character to a string
    
    return _string #returns a string of characters

Convert_Num_ascii([72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 33])

'Hello World!'

In [19]:
def Convert_Binary_String(_int):
    int_2 = bin(_int) #converts the exponent from decimal to binary
    binary_list = int_2.split('b') #splits the binary number into two elements at b
    binary_list.pop(0) # removes the first element of the list 
    bits = binary_list[0] #captures the first element in the list, which contains the binary string
    
    return bits #returns the binary number as a string

Convert_Binary_String(19)

'10011'

In [20]:
def modular_exponentiation(b, n_10 , m):
    x = 1
    power = b%m
    binary = Convert_Binary_String(n_10) #converts the integer into a binary string
    list_of_bin_digits = [] #creates an empty list
    
    #create a list that contains each of the digits in the binary number as it's own element in the list
    for i in binary:  
        list_of_bin_digits.append(i)
    list_of_bin_digits = [int(i) for i in list_of_bin_digits] #convert the list elements into integers
    list_of_bin_digits.reverse() #reverse the list_of_bin_digits, so the for loop reads from right to left of the binary number
    
    for i in list_of_bin_digits:
        if i == 1:
            x = (x * power)%m
        power = (power*power)%m 
    return x

modular_exponentiation(2, 10, 7)

2

In [24]:
#use the Euclidean Algorith to find the GCD
def Euclidean_Alg(a, b):
    while b != 0:
        r = a % b #r is the remainder of a mod b
        a = b #a is now equal to b
        b = r #b is now euqal to r
        #repeat until b is = 0
    return a #a is now the gcd of the initial inputs of a and b

Euclidean_Alg(11, 157)

1

In [25]:
def Find_Public_Key_e(p, q):
    n = p*q
    a = (p-1)*(q-1)
    
    #create a list of odd numbers to test values for e
    start, end = 2, 500 #e will be inbetween the start and end numbers
    list_of_odds = [] #create an empty list
    for num in range(start, end + 1): #iterate through the starting and ending numbers defined above
        if num % 2 != 0: #check to see if the number is odd
            list_of_odds.append(num) #if the number is odd, add it to the list
            
    #find a value for e that make sure it's relatively prime with a
    for e in list_of_odds:
        if Euclidean_Alg(e, a) == 1:
            return e, n
        
print(Find_Public_Key_e(11, 157))

(7, 1727)


In [26]:
#d will be equal to the inverse of e mod (p-1)*(q-1)
def Find_Private_Key_d(e, p, q):
    x = e
    n = p*q #will need to be returned as a part of the private key
    a = (p-1)*(q-1)
    y = a
    
    #initialize variables for the bezout coeffiecents
    oldold_s = 1
    old_s = 0
    oldold_t = 0
    old_t = 1
    
    #this while loop finds d, in addtion to finding the bezout coefficients
    while y != 0: 
        q = x // y
        r = x % y
        x = y
        y = r
        s = oldold_s - q*old_s
        t = oldold_t - q*old_t
        oldold_s = old_s
        oldold_t = old_t 
        old_s = s
        old_t = t
        
    d = oldold_s
    return d, n

print(Find_Private_Key_d(7, 11, 157))

(223, 1727)


In [27]:
def Encode(n, e, message):
    converted_message = Convert_Text_ascii(message) #convert you message into a list of ascii numbers
    cipher_text = [] #create an empty list
    for i in converted_message: #for each of the numbers in the list of ascii numbers
        c = (i**e) % n #take i to the e and mod by n
        cipher_text.append(c) #add each of the new, encoded numbers to a list

    return cipher_text

print(Encode(1727, 7, "Hello World!"))

[217, 601, 510, 510, 991, 901, 21, 991, 445, 510, 1607, 1540]


In [29]:
def Decode(n, d, cipher_text):
    decoded_number_list = [] #create an empty list
    for i in cipher_text: #loop through each number in the list
        decoded_number = (i**d)% n #take i to the d and mod by n
        decoded_number_list.append(decoded_number) #add the decoded number to a list that contains the decoded ascii numbers
    message = Convert_Num_ascii(decoded_number_list) #convert the list to of ascii numbers to a message
    
    return message

print(Decode(1727, 223, [217, 601, 510, 510, 991, 901, 21, 991, 445, 510, 1607, 1540]))

Hello World!


#### Now the fun begins:

In [38]:
def main():
    
    what_do_you_want = input("Do you want to Get Keys, Encode or Decode?")

    if what_do_you_want == "Get Keys": #to Get Keys you will need to choose two different prime numbers
        p = int(input("pick a prime"))
        q = int(input("pick another prime"))
        a = Find_Public_Key_e(p, q) #call the Find_Public_Key_e using the numbers you've entered for p and q
        b = Find_Private_Key_d(a[0], p, q) #call the Find_Private_Key_d using the numbers you've entered for p and q, and what you found for e from your Find_Public_Key_e
        print("Public Key", a, "Private Key", b)

    if what_do_you_want == "Encode": #to Encode your message you will need to choose choose your message and have your public key handy
        message = input("What is the message you'd like to encode?")
        public_key = input("What is your public key?")
        a = public_key.split(",") #the public key comes in as a string, you will need to make it into a list
        e = int(a[0]) #from your new list a, of pubilc keys you can use the index to assign the appropriate values for e and n and make them integers
        n = int(a[1])
        encoded_message = Encode(n, e, message) #call the Encode function with your values for n, e and your message to encode your function
        print(encoded_message) #this will return a list of integers
    
    if what_do_you_want == "Decode":
        coded_message = input("What is your encoded message? (please remove any brackets!!!)")
        a = coded_message.split(",") #decoded message is now a list of strings, you will need to convert the string into integrs
        coded_message_list_of_ints = []
        for i in a: #this for loop will convert each element in the list of strings into integers
            interger_values = int(i)
            coded_message_list_of_ints.append(interger_values)
        private_key = input("What is your private key?")
        b = private_key.split(",") #this will create a list with the two values entered for the private key.
        d = int(b[0]) #from your new list b, of private keys you can use the index to assign the appropriate values for e and n and make them integers
        n = int(b[1])
        decoded_message = Decode(n, d, coded_message_list_of_ints)
        print(decoded_message) #this will return your message


if __name__ == '__main__':
    main()    

Do you want to Get Keys, Encode or Decode? Decode
What is your encoded message? (please remove any brackets!!!) 217, 601, 510, 510, 991, 901, 21, 991, 445, 510, 1607, 1540
What is your private key? 223, 1727


Hello World!


### Demonstrate how your RSA works below, by encoding and decoding a sample message here using your main above and sample inputs. 


**1. To generate private and public keys, all you need is two prime numbers**

_In the main() function, when prompted, input "Get Keys" and enter two different prime numbers._


In [39]:
#make sure you've run all functions above before trying to execute any of the code below
main()

# Here is an example of the prompts and answers you can get by trying to find the keys:
# Do you want to Get Keys, Encode or Decode? Get Keys
# pick a prime 11
# pick another prime 157
# public key (7, 1727) private key (223, 1727)

Do you want to Get Keys, Encode or Decode? Get Keys
pick a prime 11
pick another prime 157


Public Key (7, 1727) Private Key (223, 1727)


**2. To encode a message, you will need your public key and your message**

* In the code below, when prompted, input "Encode" and enter values for e and n from your public key.

In [40]:
main()

# Here is an example of the prompts and answers you can get by trying to find the keys:
# Do you want to Get Keys, Encode or Decode? Encode
# What is the message you'd like to encode? Happy Halloween!
# What is your public key? 5, 3781
# [217, 601, 510, 510, 991, 901, 21, 991, 445, 510, 1607, 1540]

Do you want to Get Keys, Encode or Decode? Encode
What is the message you'd like to encode? Hello World!
What is your public key? 7, 1727


[217, 601, 510, 510, 991, 901, 21, 991, 445, 510, 1607, 1540]


**3. To decode a message, you will need your private key and your encoded message**

* In the code below, when prompted, input "Decode" and enter values for d and n from your private key.

In [ ]:
main()

# Here is an example of the prompts and answers you can get by trying to find the keys:
# Do you want to Get Keys, Encode or Decode? Decode
# What is your encoded message? (please remove any brackets!!!) 217, 601, 510, 510, 991, 901, 21, 991, 445, 510, 1607, 1540
# What is your private key? 223, 1727
# Hello World!